<a href="https://colab.research.google.com/github/isaacmaruyama/nightlights/blob/main/blackmarble_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Overview
This is a Python script I wrote that uses nightlights data taken from NASA's Black Marble program to examine trends in luminosity for specific Chinese administrative states.



In [1]:
# Import library dependencies
import osgeo.gdal as gdal, numpy as np, os, ee, geemap

# Earth Engine authentification
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=Ta68WPM0VpsXxVS5NyhRL_1yAM70d1fMZ_K_S4ALuZg&tc=20Qxzpuu4eSyFsbkai3ZGNBcffDi60sJuMX66tSKvqI&cc=Fev7AfM1tiu57wFCq1KYJWqZzTUHDH6BK5lEX1BY08c

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXng6CPiVGaBCQK_US5n8n0zLrlDwRCAd2u5h-7rE09UOgjmekntUMM

Successfully saved authorization token.


In [35]:
# Set time variables for luminosity comparison
month = '09' # 2-digit format
comparison_years = [2022, 2023] # 2 years in [old, new] format

# Create colormap based on radiance percentiles
radiance_colormap= ['#80080', '#BF0040']

# Create empty lists for comparing radiance levels
old_list = []
new_list = []

# Load a feature collection with Chinese administrative states
china_states = ee.FeatureCollection('FAO/GAUL/2015/level1') \
    .filter(ee.Filter.eq('ADM0_NAME', 'China'))

# Define style parameters for initial visualization
style_params = {'color': 'white', 'width': 1, 'fillColor': '00000000'}

# Define counter variable for loop
counter = 0

# Generate radiance data for each comparison period
for year in comparison_years:
  # Define the time range for filtering the image collection
  start_date = f'{year}-{month}-01'
  end_date = f'{year}-{month}-20'

  # Load an image collection with VIIRS monthly nightlights data
  nightlights = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG') \
      .filter(ee.Filter.date(start_date, end_date)) \
      .select('avg_rad')

  # Define visualization parameters for nightlights
  nightlight_vis_params = {'min': 0.0, 'max': 60.0}

  # Create a mosaic of nightlights and clip it to include only China for both years
  mosaic_nightlights = nightlights.mosaic()
  clipped_nightlights = mosaic_nightlights.clip(china_states)

  # Store separate mosaics for both years
  if counter == 0:
    old_nightlights = clipped_nightlights
  elif counter == 1:
    new_nightlights = clipped_nightlights

  # Reduce regions to compute the mean radiance within each Chinese state
  average_radiance = clipped_nightlights.reduceRegions(
      collection=china_states,
      reducer=ee.Reducer.mean(),
      scale=1000  # Scale can be adjusted based on data resolution
  )

  # Create a new feature collection with administrative state name and average radiance as properties
  radiance_collection = average_radiance.map(lambda feature: feature.select(**{
    'propertySelectors': ['ADM1_NAME', 'mean'],
    'newProperties': ['State Names', 'Avg Radiance']
    }))

  # Extract a list of features from the collection
  radiance_list = radiance_collection.getInfo()['features']

  # Loop through each feature and add its properties to the comparison lists
  for feature in radiance_list:
      name = feature['properties']['State Names']
      avg_rad = feature['properties']['Avg Radiance']

      # Add properties to the lists as tuples
      if counter == 0:
        old_list.append((name, avg_rad))
      elif counter == 1:
        new_list.append((name, avg_rad))

  counter += 1

# Make a list containing the change in radiance for each administrative state
difference_list = [(name, (value2 - value1)/value1) for (name, value1), (_, value2) in zip(old_list, new_list)]
print(difference_list)

[('Anhui Sheng', -0.09349946487636573), ('Beijing Shi', -0.022815718812826704), ('Chongqing Shi', 0.03904718942641669), ('Fujian Sheng', 0.09806465724124307), ('Gansu Sheng', 0.10941124429069228), ('Guangdong Sheng', 0.04782411818116257), ('Guangxi Zhuangzu Zizhiqu', 0.09661702674312798), ('Guizhou Sheng', 0.17494570270158372), ('Hainan Sheng', 0.12563631374170423), ('Hebei Sheng', 0.03286210539258551), ('Henan Sheng', -0.04586821262075667), ('Hubei Sheng', -0.01882396546120645), ('Hunan Sheng', -0.02370029272764351), ('Jiangsu Sheng', -0.02927370452385976), ('Jiangxi Sheng', 0.11729226171020815), ('Liaoning Sheng', -0.014026846953567077), ('Nei Mongol Zizhiqu', 0.05092549979450901), ('Ningxia Huizu Zizhiqu', 0.2561426162239362), ('Qinghai Sheng', 0.15896785321813064), ('Shaanxi Sheng', 0.0268346061308152), ('Shandong Sheng', 0.020014878679431997), ('Shanghai Shi', -0.019721221281452126), ('Shanxi Sheng', 0.016449584511891477), ('Sichuan Sheng', 0.6643025693356956), ('Tianjin Shi', -0.

In [48]:
# Extract radiance values from difference list
radiance_list = [entry[1] for entry in difference_list]
print(radiance_list)

radiance_sorted = np.sort(radiance_list)
radiance_quintiles = np.percentile(radiance_sorted, [20, 40, 60, 80])

print(radiance_quintiles)

def get_quintile(number):
  if number <= radiance_quintiles[0]:
    return

"""
min_value = min(radiance_list)
max_value = max(radiance_list)

for item in radiance_list:
  if item < 0:
    percentile = item / min_value
    print(percentile)


# Create list of colors
def create_custom_colormap(list):
    min_value = min(list)
    max_value = max(list)

    max_abs_value = max(abs(max(integer_list)), abs(min(integer_list)))

    # Define a custom colormap going from blue to red
    colors = [(0, 0, 1), (1, 0, 0)]  # Blue to Red

    # Map normalized values to colors using the custom colormap
    colormap_values = [(int(c[0] * 255), int(c[1] * 255), int(c[2] * 255)) for c in np.linspace(colors[0], colors[1], len(integer_list))]

    return colormap_values

colormap_values = create_custom_colormap(radiance_list)
"""

[-0.09349946487636573, -0.022815718812826704, 0.03904718942641669, 0.09806465724124307, 0.10941124429069228, 0.04782411818116257, 0.09661702674312798, 0.17494570270158372, 0.12563631374170423, 0.03286210539258551, -0.04586821262075667, -0.01882396546120645, -0.02370029272764351, -0.02927370452385976, 0.11729226171020815, -0.014026846953567077, 0.05092549979450901, 0.2561426162239362, 0.15896785321813064, 0.0268346061308152, 0.020014878679431997, -0.019721221281452126, 0.016449584511891477, 0.6643025693356956, -0.008147892923283847, 0.05992178922790568, 0.15174962739485595, 0.2738641807545528, 0.16589495508489727, 0.02078733238097829, -0.016291272474483004]
[-0.01882397  0.02078733  0.05992179  0.15174963]


'\nmin_value = min(radiance_list)\nmax_value = max(radiance_list)\n\nfor item in radiance_list:\n  if item < 0:\n    percentile = item / min_value\n    print(percentile)\n\n\n# Create list of colors\ndef create_custom_colormap(list):\n    min_value = min(list)\n    max_value = max(list)\n\n    max_abs_value = max(abs(max(integer_list)), abs(min(integer_list)))\n\n    # Define a custom colormap going from blue to red\n    colors = [(0, 0, 1), (1, 0, 0)]  # Blue to Red\n\n    # Map normalized values to colors using the custom colormap\n    colormap_values = [(int(c[0] * 255), int(c[1] * 255), int(c[2] * 255)) for c in np.linspace(colors[0], colors[1], len(integer_list))]\n\n    return colormap_values\n\ncolormap_values = create_custom_colormap(radiance_list)\n'

In [ ]:
# Create color mapping function
def create_custom_colormap(integer_list):
    max_abs_value = max(abs(max(integer_list)), abs(min(integer_list)))

    # Define a custom colormap going from blue to red
    colors = [(0, 0, 1), (1, 0, 0)]  # Blue to Red

    # Map normalized values to colors using the custom colormap
    colormap_values = [(int(c[0] * 255), int(c[1] * 255), int(c[2] * 255)) for c in np.linspace(colors[0], colors[1], len(integer_list))]

    return colormap_values

integer_list = [entry[1] for entry in difference_list]

# Create color translation function
def rgb_to_hex(rgb):
    return "#{:02x}{:02x}{:02x}".format(rgb[0], rgb[1], rgb[2])

# Create a custom colormap using the blue-to-white-to-red scheme
colormap_values = create_custom_colormap(integer_list)

hex_colors = [rgb_to_hex(rgb) for rgb in colormap_values]

# Combine the original data with the new RGB color values
result_data = [(name, value, color) for (name, value), color in zip(difference_list, hex_colors)]

print(result_data)

[('Anhui Sheng', -0.09349946487636573, '#0000ff'), ('Beijing Shi', -0.022815718812826704, '#0800f6'), ('Chongqing Shi', 0.03904718942641669, '#1100ee'), ('Fujian Sheng', 0.09806465724124307, '#1900e5'), ('Gansu Sheng', 0.10941124429069228, '#2200dd'), ('Guangdong Sheng', 0.04782411818116257, '#2a00d4'), ('Guangxi Zhuangzu Zizhiqu', 0.09661702674312798, '#3300cc'), ('Guizhou Sheng', 0.17494570270158372, '#3b00c3'), ('Hainan Sheng', 0.12563631374170423, '#4400bb'), ('Hebei Sheng', 0.03286210539258551, '#4c00b2'), ('Henan Sheng', -0.04586821262075667, '#5500aa'), ('Hubei Sheng', -0.01882396546120645, '#5d00a1'), ('Hunan Sheng', -0.02370029272764351, '#660099'), ('Jiangsu Sheng', -0.02927370452385976, '#6e0090'), ('Jiangxi Sheng', 0.11729226171020815, '#770088'), ('Liaoning Sheng', -0.014026846953567077, '#7f007f'), ('Nei Mongol Zizhiqu', 0.05092549979450901, '#880077'), ('Ningxia Huizu Zizhiqu', 0.2561426162239362, '#90006e'), ('Qinghai Sheng', 0.15896785321813064, '#990066'), ('Shaanxi S

In [41]:
print(radiance_collection.first().getInfo())

# Set to white as a placeholder before creating the colormap
difference_collection = radiance_collection.map(lambda feature: feature.set('Color', '#ffffff'))

print(difference_collection.first().getInfo())


{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[114.86630940613028, 32.98962461314836], [114.90612909862571, 32.96266046619508], [114.95810888052213, 32.95361293921174], [115.00071559689675, 32.92750046203242], [115.038354821076, 32.899234165086725], [115.09219408378587, 32.899992243143785], [115.1465594199859, 32.89113649727654], [115.17316236498162, 32.85912906609684], [115.17770178813738, 32.81488136507021], [115.17972622439297, 32.77162807732093], [115.1891259601838, 32.72936032044944], [115.2084428226344, 32.68897862262151], [115.22025944675923, 32.61940776651804], [115.27243088168684, 32.596786790845194], [115.31987132434374, 32.57762155583151], [115.3669504476962, 32.55796135782191], [115.4128479786452, 32.53602700662325], [115.44820425104346, 32.5049559844247], [115.47699662064664, 32.467927672579705], [115.50751472029654, 32.432241439687594], [115.54756639596617, 32.41791655884181], [115.58761801429755, 32.4035917457584], [115.63433152698848, 32.423836079

In [ ]:
# Create a Map instance using geemap
Map = geemap.Map(center=[35, 105], zoom=4)  # Adjust the center and zoom level as needed

Map.addLayer(clipped_nightlights, nightlight_vis_params, 'Nightlights')

# Create separate layers for each state, colored based on radiance following the previously generated colormap
for item in result_data:
    name, radiance, color = item
    selected_feature = china_states.filter(ee.Filter.eq('ADM1_NAME', name))
    Map.addLayer(selected_feature, {'color': color}, name)

Map.addLayerControl()

# View map
Map

Map(center=[35, 105], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chi…

In [ ]:
Map = geemap.Map()

legend_keys = ['One', 'Two', 'Three', 'Four', 'ect']
# colorS can be defined using either hex code or RGB (0-255, 0-255, 0-255)
legend_colors = ['#8DD3C7', '#FFFFB3', '#BEBADA', '#FB8072', '#80B1D3']
# legend_colors = [(255, 0, 0), (127, 255, 0), (127, 18, 25), (36, 70, 180), (96, 68 123)]

Map.add_legend(
    keys=legend_keys, colors=legend_colors, position='bottomleft'
)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [3]:
# Load watersheds from a data table.
sheds = ee.FeatureCollection('USGS/WBD/2017/HUC06') \
  # Filter to the continental US.
  .filterBounds(ee.Geometry.Rectangle(-127.18, 19.39, -62.75, 51.29)) \
  # Convert 'areasqkm' property from string to number.
  .map(lambda feature: feature.set('areasqkm', ee.Number.parse(feature.get('areasqkm'))))

# Display the table and print its first element.
ee.batch.Export.table.toDrive(collection=sheds, description='watersheds', fileFormat='CSV').start()
print('First watershed', sheds.first().getInfo())

# Print the number of watersheds.
print('Count:', sheds.size().getInfo())

# Print stats for an area property.
print('Area stats:', sheds.aggregate_stats('areasqkm').getInfo())


SyntaxError: ignored